# Index对象的创建,、查、改、增、删和使用
想要用好pandas，必须了解其核心对象之一的**索引**。
- 索引类似于元组，其本身是不能修改的；
- 其在数据进行整体运算时，辅助自动对齐，这是pandas不同于其他数据处理库的一大方面；
- 多层索引可以帮助改变表的形态，如透视表等。

所以，这一章要仔细学习。

In [1]:
!cd

E:\ML\实战\pandas实用教程


In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

# 引言 Index
Index对象保存着索引标签数据，它可以快速找到标签对应的整数下标，其功能与Python的字典类似。

In [3]:
dict1={"Province":["Guangdong","Beijing","Qinghai","Fujiang"],
      "year":[2018]*4,
      "pop":[1.3,2.5,1.1,0.7]}

df1=DataFrame(dict1)
df1

,Province,pop,year
0,Guangdong,1.3,2018
1,Beijing,2.5,2018
2,Qinghai,1.1,2018
3,Fujiang,0.7,2018


调用.columns返回DataFrame对象的列索引（即所有列标签）

In [4]:
col_index=df1.columns
col_index.values

array(['Province', 'pop', 'year'], dtype=object)

调用.index返回DataFrame对象的列索引

In [8]:
ind_index=df1.index
ind_index.values

array([0, 1, 2, 3], dtype=int64)

Index对象可当做一维数组，适合Numpy数组的下标运算，但Index对象只是可读，创建后不可修改。

In [9]:
print(col_index[[1,2]])
print(ind_index[ind_index>1])

Index(['pop', 'year'], dtype='object')
Int64Index([2, 3], dtype='int64')


index对象具有字典的映射功能，.get_loc(value)获得单值得下标，.get_indexer(values)获得一组值得下标，当值不存在则返回-1：

In [10]:
print(col_index.get_loc('pop'))
print(col_index.get_indexer(['pop','year']))

1
[1 2]


Index对象调用Index()来创建，可传递给DataFrame对象的参数index和columns。因为Index是不可变的，因此多个DataFrame对象的索引可以是同个Index对象。

In [11]:
index=pd.Index(['a','b','c'])
df2=DataFrame(np.random.randint(1,10,(3,3)),index=index,columns=index)
df2

,a,b,c
a,5,7,2
b,7,7,6
c,5,5,5


---
# 1. 单层索引

## 1.1 创建
**pd.Index(data, dtype = Object, name = None)**
- data：一维列表
- dtype：索引元素的类型，默认为object型
- name：索引的名字，类似于列的名字

In [2]:
data = ['a','b','c']
index = pd.Index(data, name = 'name1')
index

Index(['a', 'b', 'c'], dtype='object', name='name1')

#### 从返回值可以看到，index由三部分组成，可以分别查看。

In [52]:
index.name

'name1'

In [53]:
index.values

array(['a', 'b', 'c'], dtype=object)

In [54]:
index.dtype

dtype('O')

---
## 1.2 查
- 由于索引本身是不可更改的，所以只有查询操作；
- 查询方式和一维ndarray或Series的.iloc[]完全一样。

In [7]:
index[0] # scalar，返回值

'a'

In [8]:
index[0:2] # 范围，返回index

Index(['a', 'b'], dtype='object', name='name1')

In [9]:
index[[0,2]] # 列表，返回index

Index(['a', 'c'], dtype='object', name='name1')

In [10]:
mask = [True,False,True]  # mask,返回index
index[mask]

Index(['a', 'c'], dtype='object', name='name1')

---
## 1.3 改索引名
虽然索引的值是不能修改的，但是名字确是可以修改的。

### 1.3.1 直接改

In [11]:
index.name = 'new_name' 
index

Index(['a', 'b', 'c'], dtype='object', name='new_name')

### 1.3.2 函数改
**Index.set_names(names, inplace=False)**
- names：要设置的名字，可以为名字的列表；
- inplace：是否原地修改。

In [55]:
index.set_names('new_name')

Index(['a', 'b', 'c'], dtype='object', name='new_name')

---
## 1.4 增

### 1.4.1 按位置添加一行
**Index.insert(loc, value)**
- loc：位置编号
- value：值

In [67]:
index

Index(['a', 'b', 'c'], dtype='object', name='name1')

In [68]:
index.insert(1,'d')

Index(['a', 'd', 'b', 'c'], dtype='object', name='name1')

### 1.4.2 尾部添加多行
**Index.append(other)**
- other：其他索引对象

In [59]:
index1 = index.copy()
index1

Index(['a', 'b', 'c'], dtype='object', name='name1')

In [ ]:
index1.append( index)

### 1.4.2 并
**Index.union(other)**

In [62]:
index2 = pd.Index(['b','c','d'])
index2

Index(['b', 'c', 'd'], dtype='object')

In [ ]:
index1.union(index2)

----
## 1.5 删

### 1.5.1 按位置删除一行
**Index.delete(loc)**
- loc：位置编号

In [63]:
index1.delete(1)

Index(['a', 'c'], dtype='object', name='name1')

### 1.5.2 按索引删除多行
**Index.drop(labels)**
- labels：索引列表

In [66]:
index1.drop(['a','b'])

Index(['c'], dtype='object', name='name1')

### 1.5.3 交
**Index.intersection(other)**

In [69]:
index1.intersection(index2) 

Index(['b', 'c'], dtype='object')

---
# 2. 多层索引

In [12]:
ser_obj = pd.Series(np.random.randn(12),
                    index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd', 'd', 'd'],
                           [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])

ser_obj

a  0   -0.897226
   1    0.888239
   2    0.488880
b  0   -1.876541
   1   -0.619938
   2   -0.447302
c  0   -0.851257
   1   -0.930598
   2   -0.373277
d  0    0.618972
   1    0.554697
   2   -0.964972
dtype: float64

MultiIndex索引对象

打印这个Series的索引类型，显示是MultiIndex
直接将索引打印出来，可以看到有lavels,和labels两个信息。levels表示两个层级中分别有那些标签，labels是每个位置分别是什么标签。

In [13]:
print(type(ser_obj.index))

print(ser_obj.index)

<class 'pandas.core.indexes.multi.MultiIndex'>
MultiIndex(levels=[['a', 'b', 'c', 'd'], [0, 1, 2]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])


选取子集

根据索引获取数据。因为现在有两层索引，当通过外层索引获取数据的时候，可以直接利用外层索引的标签来获取。
当要通过内层索引获取数据的时候，在list中传入两个元素，前者是表示要选取的外层索引，后者表示要选取的内层索引。

In [14]:
# 外层选取
print(ser_obj['c'])

0   -0.851257
1   -0.930598
2   -0.373277
dtype: float64


In [15]:
# 内层选取
print(ser_obj[:, 1])

a    0.888239
b   -0.619938
c   -0.930598
d    0.554697
dtype: float64


In [16]:
# 外层+内层选取
print(ser_obj['c', 2])
print(type(ser_obj['c', 2]))

-0.373277148194
<class 'numpy.float64'>


.swaplevel( )交换内层与外层索引

In [17]:
print(ser_obj.swaplevel())

0  a   -0.897226
1  a    0.888239
2  a    0.488880
0  b   -1.876541
1  b   -0.619938
2  b   -0.447302
0  c   -0.851257
1  c   -0.930598
2  c   -0.373277
0  d    0.618972
1  d    0.554697
2  d   -0.964972
dtype: float64


.sortlevel( )先对外层索引进行排序，再对内层索引进行排序，默认是升序。

In [18]:
print(ser_obj.swaplevel().sortlevel())

0  a   -0.897226
   b   -1.876541
   c   -0.851257
   d    0.618972
1  a    0.888239
   b   -0.619938
   c   -0.930598
   d    0.554697
2  a    0.488880
   b   -0.447302
   c   -0.373277
   d   -0.964972
dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level=...)
  """Entry point for launching an IPython kernel.


## 2.1 创建
**pd.MultiIndex.from_tuples(labels, names = None)**
- labels：元组或列表的列表；
- names：名字的列表。

创建有多层索引的series

下面创建一个Series， 在输入索引Index时，输入了由两个子list组成的list，第一个子list是外层索引，第二个list是内层索引。

In [19]:
# data = [['a','one'],['a','two'],['b','one']]
data = [('a','one'),('a','two'),('b','one')]
index = pd.MultiIndex.from_tuples( data, names = ['name1','name2'])
index

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 0, 1], [0, 1, 0]],
           names=['name1', 'name2'])

In [14]:
s = pd.Series([1,2,3], index = index)
s

name1  name2
a      one      1
       two      2
b      one      3
dtype: int64

---
## 2.2 查
- 查询方法和单层索引完全一致。

In [15]:
index[0]  # scalar，返回值

('a', 'one')

In [16]:
index[0:2]  # 范围，返回MultiIndex

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 0], [0, 1]],
           names=['name1', 'name2'])

In [17]:
index[[0,2]] # 列表，返回MultiIndex

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 1], [0, 0]],
           names=['name1', 'name2'])

In [18]:
mask = [True,False, True] # mask，返回MultiIndex
index[mask]

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 1], [0, 0]],
           names=['name1', 'name2'])

**获取某一层索引：MultiIndex.get_level_values(level)**
- level：int，选中的那一层

In [20]:
index.get_level_values(0)

Index(['a', 'a', 'b'], dtype='object', name='name1')

In [21]:
index.get_level_values(1)

Index(['one', 'two', 'one'], dtype='object', name='name2')

## 2.3 改

### 2.3.1 改索引名（函数改）
**MultiIndex.set_names(names, level=None, inplace=False)**
- names：要设置的名字，可以为名字的列表；
- level：多层索引需要设置修改的索引层次，可以为列表，要与names匹配；
- inplace：是否原地修改。

In [22]:
index.set_names('new_name_1',level=0)

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 0, 1], [0, 1, 0]],
           names=['new_name_1', 'name2'])

### 2.3.2 改索引层次顺序
**MultiIndex.swaplevel(i=-2, j=-1)**
- 改变level i 和level j的次序

In [23]:
index.swaplevel()

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 1, 0], [0, 0, 1]],
           names=['name2', 'name1'])

**Series.swaplevel(i=-2, j=-1)`**

**DataFrame.swaplevel(i=-2, j=-1, axis = 1)`**

- axis：0-行索引，1-列索引。
这两个函数更实用一些。

In [26]:
index.swaplevel()

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 1, 0], [0, 0, 1]],
           names=['name2', 'name1'])

In [27]:
columns = index.copy()
columns.set_names( names = ['name3','name4'], level = [0,1], inplace = True) #列索引取和行索引相同，只是改了名字
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], index= index, columns = columns)
df

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [28]:
df.swaplevel( axis =1)  # 交换列索引顺序

name4       one two one
name3         a   a   b
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

---
# 3. 多层索引使用方法
当对values进行查看时，多层索引可以分开使用。

In [29]:
df1 = df.copy()
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

索引为空不代表缺失，缺省写法，意思是之前的索引一致。

## 3.1 对于外层索引
**记住：**
- 无论是Series还是DataFrame，外层索引都是可以直接使用，也就是说可以认为只有这一层索引；
- **用法和第二篇查、改、增、删提到的方法完全相同**。

### 3.1.1 `[]`
快捷操作，还是四种用法。

In [27]:
df1['b']  # 列外层

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [28]:
df1[['a','b']] # 列外层

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [29]:
df1[0:2]  # 行外层

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6

In [30]:
mask =[True,False,True]  # 行外层
df1[mask]

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
b     one     7   8   9

### 3.1.2  `.loc[]`

**下面都以第一维度为例，第二维可以类比。**

In [31]:
df1.loc['a','b']  # 单行索引'a' 

name4,one
name2,
one,3
two,6


In [32]:
df1.loc['a':'b','b']   #范围

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [33]:
df1.loc[['a','b'],'b'] #列表

name4        one
name1 name2     
a     one      3
      two      6
b     one      9

In [34]:
mask = [True, False, True] # mask
df1.loc[mask,'b']

,name4,one
name1,name2,
a,one,3
b,one,9


### 3.1.3 `.iloc[]`
**下面都以第一维度为例，第二维可以类比。**

In [35]:
df1.iloc[0,0:2] 

name3  name4
a      one      1
       two      2
Name: (a, one), dtype: int64

In [36]:
df1.iloc[0:2,0:2] 

name3         a    
name4       one two
name1 name2        
a     one     1   2
      two     4   5

In [37]:
df1.iloc[[0,1],0:2] 

name3         a    
name4       one two
name1 name2        
a     one     1   2
      two     4   5

In [38]:
mask = [True, False, True]
df1.iloc[mask,0:2]

name3         a    
name4       one two
name1 name2        
a     one     1   2
b     one     7   8

## 3.2 对于内层索引
- **内层索引不可直接使用，必须先外层、再内层，直接使用会报错；**
- 内层只能使用单索引形式，其他形式报错。

### 3.2.1 `[ , ]`
快捷操作，只有一种用法。

In [39]:
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [40]:
df1['a','one']  #取一列，先外层单列索引，再内层单列索引，其他形式都报错

name1  name2
a      one      1
       two      4
b      one      7
Name: (a, one), dtype: int64

### 3.2.2 `.loc[ , ]`

In [41]:
df1.loc['a','one'] # 取一行，先外层单行索引，再内层单列索引，其他形式都报错

name3  name4
a      one      1
       two      2
b      one      3
Name: (a, one), dtype: int64

### 3.2.3 `.iloc[ , ]`
这种方法不受影响，因为`.iloc[]`无视索引，只按照位置定位，所以和3.1.3节外层索引部分完全相同。

## 3.3 xs直接选取法
适合在单层level选取，不能行列同时操作。
**`Series.xs(key, level=None, drop_level=True)`**
** `DataFrame.xs(key, axis=0, level=None, drop_level=True)`**
- key: 要选取的索引值或其列表；
- axis：0-行索引，1-列索引；
- level：索引层次；
- drop_level：True or False，是否显示用于选取的level索引，默认不显示。

In [42]:
df1 = df.copy()
df1

name3         a       b
name4       one two one
name1 name2            
a     one     1   2   3
      two     4   5   6
b     one     7   8   9

In [43]:
df1.xs( 'one', axis = 0, level = 1 )  # 行索引的level 1, 有两行

name3   a       b
name4 one two one
name1            
a       1   2   3
b       7   8   9

In [47]:
df1.xs( 'two', axis = 1, level = 1 ) # 列索引的level 1，有一列

name3        a
name1 name2   
a     one    2
      two    5
b     one    8